In [2]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.functional as F
import torch.nn.functional as F
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import graphviz
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
# Descargar los recursos necesarios
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

import re

[nltk_data] Downloading package punkt to
[nltk_data]     /home/miguelfsoto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/miguelfsoto/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/miguelfsoto/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/miguelfsoto/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/miguelfsoto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
nombre_archivo = 'test_text.txt'

# Abrir el archivo en modo lectura
with open(nombre_archivo, 'r') as archivo:
    # Leer el contenido del archivo
    corpus = archivo.read()

# Convertir el contenido a minúsculas y eliminar tildes
corpus = unidecode(corpus.lower())
# Eliminar números y caracteres especiales usando expresiones regulares
corpus = re.sub(r'[^a-z\s]', '', corpus)
corpus = re.sub(r'\s+', ' ', corpus)

# Imprimir el contenido modificado
print(corpus)

in geometry a square is a twodimensional plane figure with four equal sides and all the four angles are equal to ninety degrees the properties of rectangle are somewhat similar to a square but the difference between the two is a rectangle has only its opposite sides equal therefore a rectangle is called a square only if all its four sides are of equal length a square is a twodimensional closed shape with equal sides and vertices its opposite sides are parallel to each other we can also think of a square as a rectangle with equal length and breadth a flat shape with four sides of equal length and four angles of ninety degrees a square is a figure with four equal sides and four right angles a plane rectangle with four equal sides and four right angles a foursided regular polygon a square is a shape with four sides that are all the same length and four corners that are all right angles a flat shape with straight sides where all sides have equal length and every interior angle is a right a

In [4]:
## Tokenizar el corpus
## whitout stopwords
stop_words = set(stopwords.words('english'))
stop_words.update(['us','whose'])
token_corpus = word_tokenize(corpus)
token_corpus = [word for word in token_corpus if not word in stop_words]
print(token_corpus)

['geometry', 'square', 'twodimensional', 'plane', 'figure', 'four', 'equal', 'sides', 'four', 'angles', 'equal', 'ninety', 'degrees', 'properties', 'rectangle', 'somewhat', 'similar', 'square', 'difference', 'two', 'rectangle', 'opposite', 'sides', 'equal', 'therefore', 'rectangle', 'called', 'square', 'four', 'sides', 'equal', 'length', 'square', 'twodimensional', 'closed', 'shape', 'equal', 'sides', 'vertices', 'opposite', 'sides', 'parallel', 'also', 'think', 'square', 'rectangle', 'equal', 'length', 'breadth', 'flat', 'shape', 'four', 'sides', 'equal', 'length', 'four', 'angles', 'ninety', 'degrees', 'square', 'figure', 'four', 'equal', 'sides', 'four', 'right', 'angles', 'plane', 'rectangle', 'four', 'equal', 'sides', 'four', 'right', 'angles', 'foursided', 'regular', 'polygon', 'square', 'shape', 'four', 'sides', 'length', 'four', 'corners', 'right', 'angles', 'flat', 'shape', 'straight', 'sides', 'sides', 'equal', 'length', 'every', 'interior', 'angle', 'right', 'angle', 'ninety

In [5]:
pos_tags = nltk.pos_tag(token_corpus)
# Crear una instancia del lematizador
lemmatizer = WordNetLemmatizer()

lemmas = []
for token, pos in pos_tags:
    # Convertir la etiqueta POS a un formato reconocible por WordNetLemmatizer
    pos = pos[0].lower() if pos[0].lower() in ['a', 'r', 'n', 'v'] else 'n'
    lemma = lemmatizer.lemmatize(token, pos=pos)
    lemmas.append(lemma)

print(lemmas)

['geometry', 'square', 'twodimensional', 'plane', 'figure', 'four', 'equal', 'side', 'four', 'angle', 'equal', 'ninety', 'degree', 'property', 'rectangle', 'somewhat', 'similar', 'square', 'difference', 'two', 'rectangle', 'opposite', 'side', 'equal', 'therefore', 'rectangle', 'call', 'square', 'four', 'side', 'equal', 'length', 'square', 'twodimensional', 'close', 'shape', 'equal', 'side', 'vertex', 'opposite', 'side', 'parallel', 'also', 'think', 'square', 'rectangle', 'equal', 'length', 'breadth', 'flat', 'shape', 'four', 'side', 'equal', 'length', 'four', 'angle', 'ninety', 'degree', 'square', 'figure', 'four', 'equal', 'side', 'four', 'right', 'angle', 'plane', 'rectangle', 'four', 'equal', 'side', 'four', 'right', 'angle', 'foursided', 'regular', 'polygon', 'square', 'shape', 'four', 'side', 'length', 'four', 'corner', 'right', 'angle', 'flat', 'shape', 'straight', 'side', 'side', 'equal', 'length', 'every', 'interior', 'angle', 'right', 'angle', 'ninety', 'degree', 'square', 'cl

In [6]:
## create dictionary
# create a list of words
vocabulary=[]
for word in lemmas:
    if word not in vocabulary:
        vocabulary.append(word)
print(vocabulary)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}


['geometry', 'square', 'twodimensional', 'plane', 'figure', 'four', 'equal', 'side', 'angle', 'ninety', 'degree', 'property', 'rectangle', 'somewhat', 'similar', 'difference', 'two', 'opposite', 'therefore', 'call', 'length', 'close', 'shape', 'vertex', 'parallel', 'also', 'think', 'breadth', 'flat', 'right', 'foursided', 'regular', 'polygon', 'corner', 'straight', 'every', 'interior', 'closed', 'euclidean', 'quadrilateral', 'mean', 'pi', 'radian', 'define', 'adjacent', 'internal', 'central', 'external', 'diagonal', 'make', 'special', 'kind', 'equilateral', 'one', 'parallelogram', 'equiangular', 'ax', 'symmetry', 'finite', 'bisection', 'result', 'triangle', 'area', 'relation', 'derive', 'algebraic', 'use', 'term', 'denote', 'product', 'multiply', 'expression', 'measure', 'must', 'thing', 'adequate', 'say', 'long', 'numerous', 'item', 'around', 'distinguish', 'inner', 'form', 'indicate', 'twlength', 'pair', 'line', 'type', 'word', 'perpendicular', 'closely', 'related', 'class', 'especia

In [7]:
frecuencia = FreqDist(token_corpus)
# Crear un diccionario para almacenar las frecuencias
frequency_word = {}
# Guardar las frecuencias en el diccionario
for palabra, frec in frecuencia.items():
    frequency_word[palabra] = frec

# Imprimir el diccionario de frecuencias
print(frequency_word)

{'geometry': 4, 'square': 36, 'twodimensional': 8, 'plane': 4, 'figure': 6, 'four': 35, 'equal': 33, 'sides': 36, 'angles': 22, 'ninety': 12, 'degrees': 10, 'properties': 2, 'rectangle': 12, 'somewhat': 2, 'similar': 2, 'difference': 2, 'two': 7, 'opposite': 5, 'therefore': 2, 'called': 1, 'length': 13, 'closed': 6, 'shape': 15, 'vertices': 1, 'parallel': 6, 'also': 2, 'think': 1, 'breadth': 1, 'flat': 3, 'right': 13, 'foursided': 3, 'regular': 6, 'polygon': 4, 'corners': 1, 'straight': 3, 'every': 3, 'interior': 3, 'angle': 7, 'euclidean': 1, 'quadrilateral': 8, 'means': 1, 'degree': 1, 'pi': 1, 'radian': 1, 'defined': 1, 'adjacent': 4, 'internal': 2, 'central': 1, 'external': 1, 'diagonals': 3, 'made': 2, 'special': 2, 'kind': 2, 'equilateral': 2, 'one': 3, 'parallelogram': 2, 'equiangular': 1, 'axes': 1, 'symmetry': 2, 'finite': 1, 'bisection': 1, 'diagonal': 1, 'results': 2, 'triangles': 1, 'side': 1, 'area': 1, 'squared': 1, 'relation': 1, 'derived': 1, 'algebraic': 2, 'use': 1, '

In [8]:
window_size = 2
idx_pairs = []
# for corpus in token_corpus:
for center_word_pos in range(len(lemmas)):
    for w in range(-window_size, window_size + 1):
        context_word_pos = center_word_pos + w
        if context_word_pos < 0 or context_word_pos >= len(lemmas) or center_word_pos == context_word_pos:
            continue
        context_word_idx = word2idx[lemmas[context_word_pos]]
        center_word_idx = word2idx[lemmas[center_word_pos]]
        idx_pairs.append((center_word_idx, context_word_idx))

print(idx_pairs)

[(0, 1), (0, 2), (1, 0), (1, 2), (1, 3), (2, 0), (2, 1), (2, 3), (2, 4), (3, 1), (3, 2), (3, 4), (3, 5), (4, 2), (4, 3), (4, 5), (4, 6), (5, 3), (5, 4), (5, 6), (5, 7), (6, 4), (6, 5), (6, 7), (6, 5), (7, 5), (7, 6), (7, 5), (7, 8), (5, 6), (5, 7), (5, 8), (5, 6), (8, 7), (8, 5), (8, 6), (8, 9), (6, 5), (6, 8), (6, 9), (6, 10), (9, 8), (9, 6), (9, 10), (9, 11), (10, 6), (10, 9), (10, 11), (10, 12), (11, 9), (11, 10), (11, 12), (11, 13), (12, 10), (12, 11), (12, 13), (12, 14), (13, 11), (13, 12), (13, 14), (13, 1), (14, 12), (14, 13), (14, 1), (14, 15), (1, 13), (1, 14), (1, 15), (1, 16), (15, 14), (15, 1), (15, 16), (15, 12), (16, 1), (16, 15), (16, 12), (16, 17), (12, 15), (12, 16), (12, 17), (12, 7), (17, 16), (17, 12), (17, 7), (17, 6), (7, 12), (7, 17), (7, 6), (7, 18), (6, 17), (6, 7), (6, 18), (6, 12), (18, 7), (18, 6), (18, 12), (18, 19), (12, 6), (12, 18), (12, 19), (12, 1), (19, 18), (19, 12), (19, 1), (19, 5), (1, 12), (1, 19), (1, 5), (1, 7), (5, 19), (5, 1), (5, 7), (5, 6),

In [9]:
'''
## create an array of word pairs from sentences and the window size
## this array is called idx_pairs

window_size = 2
idx_pairs = []
# for each sentence
for sentence in token_corpus:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

'''

'\n## create an array of word pairs from sentences and the window size\n## this array is called idx_pairs\n\nwindow_size = 2\nidx_pairs = []\n# for each sentence\nfor sentence in token_corpus:\n    indices = [word2idx[word] for word in sentence]\n    # for each word, threated as center word\n    for center_word_pos in range(len(indices)):\n        # for each window position\n        for w in range(-window_size, window_size + 1):\n            context_word_pos = center_word_pos + w\n            # make soure not jump out sentence\n            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:\n                continue\n            context_word_idx = indices[context_word_pos]\n            idx_pairs.append((indices[center_word_pos], context_word_idx))\n\nidx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array\n\n'

In [10]:
# this function return the frequency of each pair of word in the corpus

def get_frequece_array(A):
    unique_rows, counts = np.unique(A, axis=0, return_counts=True)
    #print(unique_rows)
    #print(counts)
    counts = counts.reshape(-1, 1)
    #print(counts)
    row_fre = np.concatenate((unique_rows, counts), axis=1)
    #print(row_fre)
    #print(-row_fre[:,2])
    # ind_array = np.argsort(-row_fre[:, 2])
    # print(ind_array)
    # row_fre = row_fre[ind_array]
    row_fre = np.array(sorted(row_fre, key=lambda x: -x[2]))
    return row_fre
row_fre=get_frequece_array(idx_pairs)
print(row_fre)

vocabulary_size = len(vocabulary)
print(vocabulary_size)
# Guardar la variable en un archivo CSV
np.savetxt('conexion.csv',  row_fre, delimiter=',', fmt='%d')

[[  5   7  29]
 [  7   5  29]
 [  6   7  26]
 ...
 [101  16   1]
 [101  29   1]
 [101  48   1]]
102


In [11]:
#here I generate a matrix of conectivity between words

def get_connectivity_matrix(row_fre):
    vocsize = len(row_fre)
    print(vocsize)
    connectivity_matrix = np.zeros((vocabulary_size, vocabulary_size))
    for i in range(len(row_fre)):
        connectivity_matrix[row_fre[i][0], row_fre[i][1]] = row_fre[i][2]
    return connectivity_matrix
matrix_connection_weight=get_connectivity_matrix(row_fre)

886


In [23]:
freqPairs = row_fre
indexWord = idx2word

dataframe = pd.DataFrame(freqPairs)
dataframe.columns = ['A', 'B', 'Weight']
dataframe['wordA'] = dataframe['A'].map(indexWord)
dataframe['wordB'] = dataframe['B'].map(indexWord)

dot = graphviz.Graph('G', engine='sfdp')
graph = nx.Graph()

# Create the data frame 
for _, row in dataframe.iterrows():
    a = row['A']
    b = row['B']
    w = row['Weight']
    print(w)
    graph.add_edge(a, b, weight = w)

    dot.node(str(row['A']), row['wordA'])
    dot.node(str(row['B']), row['wordB'])
    dot.edge(str(row['A']), str(row['B']), label = str(w))

# Obtener las posiciones de los nodos para el gráfico
positions = nx.spring_layout(graph)

# Obtener los pesos de las aristas
weights = nx.get_edge_attributes(graph, 'weight')

# Dibujar el grafo
# nx.draw_networkx(grafo, posiciones)
# nx.draw_networkx_edge_labels(grafo, posiciones, edge_labels=pesos)
# plt.axis('on')
# plt.show()
print(dot.source)
dot.view()

29
29
26
26
23
23
16
16
13
13
12
12
12
11
11
10
10
9
9
9
9
9
9
8
8
8
8
8
8
7
7
7
7
7
7
7
7
6
6
6
6
6
6
6
6
6
6
6
6
6
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1

'G.gv.pdf'

In [12]:
for node in grafo.nodes:
    shortest_path = nx.shortest_path(grafo, source=node, weight='weight')
    degree_centrality = nx.degree_centrality(grafo)
    betweenness_centrality = nx.betweenness_centrality(grafo, weight='weight')
    closeness_centrality = nx.closeness_centrality(grafo, distance='weight')
    clustering_coefficient = nx.clustering(grafo, weight='weight')

  

In [13]:
df_metrics = pd.DataFrame(idx2word.items(), columns=['Node', 'word'])
df_metrics['frequency_word'] = df_metrics['word'].map(frequency_word)
df_metrics['shortest_path'] = df_metrics['Node'].map(shortest_path)
df_metrics['degree_centrality'] = df_metrics['Node'].map(degree_centrality)
df_metrics['betweenness_centrality'] = df_metrics['Node'].map(betweenness_centrality)
df_metrics['closeness_centrality'] = df_metrics['Node'].map(closeness_centrality)
df_metrics['clustering_coefficient'] = df_metrics['Node'].map(clustering_coefficient)
df_metrics['eigenvector_centrality'] = df_metrics['Node'].map(nx.eigenvector_centrality(grafo, weight='weight'))
df_metrics = df_metrics.sort_values("eigenvector_centrality", ascending=False)


def class_quartiles(df,name):
    name1 = 'Class_' + name + 'low'
    name2 = 'Class_' + name + 'medium'
    name3 = 'Class_' + name + 'high'
    quartiles = np.percentile(df[name], [25, 50, 75])
    df[name1] = np.where(df[name] >= quartiles[2], 1, 0)
    df[name2] = np.where((df[name] > quartiles[0]) & (df[name] < quartiles[2]), 1, 0)
    df[name3] = np.where((df[name] <= quartiles[0]) | (df[name] >= quartiles[2]), 1, 0)
    return df

df_metrics=class_quartiles(df_metrics,'degree_centrality')
df_metrics=class_quartiles(df_metrics,'betweenness_centrality')
df_metrics=class_quartiles(df_metrics,'closeness_centrality')
df_metrics=class_quartiles(df_metrics,'clustering_coefficient')
df_metrics=class_quartiles(df_metrics,'eigenvector_centrality')


## save the metrics in a csv file
df_metrics.to_csv('metrics.csv', index=False)

## create the dataframe only with the word and all columns beging Class

df_metrics_class=df_metrics.filter(regex='Class', axis=1)

# df_metric aggregate the metrics of each word in THE FIRST COLUMN
df_metrics_class['word']=df_metrics['word']

columnas = df_metrics_class.columns.tolist()
columnas = [columnas[-1]] + columnas[:-1]
df_metrics_class = df_metrics_class[columnas]

# save the dataframe in a csv file
df_metrics_class.to_csv('metrics_class.csv', index=False)














/tmp/ipykernel_23040/1595156181.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metrics_class['word']=df_metrics['word']
